### Visualization fo self-reports ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.data_reader_popane as dr_p
import tools.display_tools as dt
import train

c:\Users\zhoux\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# APD
importlib.reload(dr_a)
importlib.reload(train)


threshold = "fixed"
participant_file = os.path.join(dr_a.Paths.DATA_DIR, "participants_details.csv")
df = pd.read_csv(participant_file)
labels = [
    "Baseline_SUDS", "BugBox_Relax_SUDS", "BugBox_Preparation_SUDS", "BugBox_Exposure_SUDS", "BugBox_Exposure_SUDS", 
    "Speech_Relax_SUDS", "Speech_SUDS", "Speech_Exposure_SUDS", "Speech_Break_SUDS"
]
suds_labels = df.loc[:, ["Participant"] + labels]
mean_suds = np.mean(suds_labels.loc[:, labels])

ha_suds_df = suds_labels.loc[suds_labels['Participant'].isin(dr_a.Groups.ha_participant_indices)]
la_suds_df = suds_labels.loc[suds_labels['Participant'].isin(dr_a.Groups.la_participant_indices)]

print(ha_suds_df)

   Participant  Baseline_SUDS  BugBox_Relax_SUDS  BugBox_Preparation_SUDS  \
3           P4             20                  0                       30   
5           P6             20                  0                       10   
6           P7             20                  0                       30   
7           P8             20                  0                       20   
9          P10             10                  0                        0   
11         P12             70                 10                       60   
14         P15             10                 10                       40   
15         P16              0                  0                       80   
17         P18             40                 10                       20   
21         P22             30                 10                       40   
25         P26              0                  0                       10   
26         P27             40                  0                       70   

In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
